# Step 1 - Prepare

As this work builds on previous work (in the Person-first/Identity-first repo) the preparation phase is relatively straightforward. Mostly, it involves importing a large .csv file with lots of columns, one of which is 'year' and one of which is 'text'. 

## Get ready

All of my jupyter notebooks begin with some code cells focussed on downloading/importing necessary packages, loading useful short names, and so forth. 

I also like to check the relevant file locations before importing the .csv files to work on. 

In [1]:
%%capture                         
                                  # The above capture statement is optional. 
                                  # You can remove this to see the chatter normally produced during import steps. 

import os                         # os is a module for navigating your machine (e.g., file directories).

import pandas as pd               # pandas is necessary for working with data frames - shortening it to pd just saves time. 
pd.set_option('display.max_colwidth', 200)   # some of the files are big so set a big column width. 
import numpy as np                # like pandas, numpy is useful and useful to have a short name for
import statistics                 # gotsta have stats

import csv                        # csv is for importing and working with csv files

import re                         # things we need for RegEx corrections
import string 
import math 




## Import the data

Having got all the packages we need and having checked the files, let's import the data by reading it in from .csv. 

In [99]:
print(os.listdir("..\\results")  )                                # check 'results' folder is not empty/has correct stuff

['.ipynb_checkpoints', 'batch_2001.csv', 'batch_2002.csv', 'batch_2003.csv', 'batch_2004.csv', 'batch_2005.csv', 'batch_2006.csv', 'batch_2007.csv', 'batch_2008.csv', 'batch_2009.csv', 'batch_2010.csv', 'batch_2011.csv', 'batch_2012.csv', 'batch_2013.csv', 'batch_2014.csv', 'batch_2015.csv', 'batch_2016.csv', 'batch_2017.csv', 'batch_2018.csv', 'batch_2019.csv', 'batch_2020.csv', 'batch_2021.csv']


In [98]:
texts_by_year = pd.read_csv('..\\results\\texts_by_year.csv')    # read in the file
texts_by_year.head(5)                                            # have a look at the top 5 rows

FileNotFoundError: [Errno 2] No such file or directory: '..\\results\\texts_by_year.csv'

## Check, clean and divide the data by year

Having got all the packages we need and having checked the files, let's import them. This requires:
* reading in and checking the .csv file
* dropping all the columns except for 'Text' and 'Year'
* checking again, see that 'Year' is appearing as a float when it shouldn't
* retyping 'Year' as integer

In [5]:
list(texts_by_year.columns)                             # Get the column names in a list to make it easier to remove the 
                                                        # ones that we don't need to keep

['Unnamed: 0',
 'Title',
 'Session_Code',
 'Author',
 'Affiliations',
 'Text',
 'Year',
 'Authors_and_Affiliations',
 'Email']

In [8]:
texts_year_only = texts_by_year.drop(['Unnamed: 0',     # Copy/paste the output from the previous step here, making sure to
                    'Title',                            # remove the columns that we want to KEEP
                    'Session_Code', 
                    'Author', 
                    'Affiliations', 
                    'Authors_and_Affiliations', 
                    'Email'], axis=1)
texts_year_only.head(5)                                         # check it again

,Text,Year
0,".-(TenthousandEuros,orequivalentvalueinkind)peryearpercompanyareconsidered â\x80\x9cModestâ\x80\x9d. Contributions above EUR 10 000.- per year are considered â\x80\x9cSignificantâ\x80\x9d. Oral pr...",2018.0
1,mentioned more than 60/169 reasons. The reasons could The ethics of clinical applications of germline genome becategorisedinto:(i)qualityoflifeofaffectedindividuals; modification:a systematic rev...,2018.0
2,implications. Asystematicoverviewofreasonsforbeingin Enablinginformed opinionsabout germline editingamong favouroragainstgermlinegenomemodificationismissing. the general public: a pilot study We ...,2018.0
3,measurement this ratio flipped. The PRISM-IMPACT study: What are the hopes and Discussion/conclusion: Our pilot study demonstrates a expectations of families and health care professionals signifi...,2018.0
4,"Cancer Centre, Sydney Childrenâ\x80\x99s Hospital, Randwick, NSW, Informedconsentinahumangermlinegeneeditingstudy- Australia, 3Hereditary Cancer Centre, Prince of Wales ethical issues Hospital, S...",2018.0


In [9]:
texts_year_only['Year'] = texts_year_only['Year'].astype(int)   # I noticed years are appearing as floats, e.g. "2004.0"
                                                                # so save the 'Year' column over itself, but as an integer
texts_year_only.head(5)                                                 # check again

,Text,Year
0,".-(TenthousandEuros,orequivalentvalueinkind)peryearpercompanyareconsidered â\x80\x9cModestâ\x80\x9d. Contributions above EUR 10 000.- per year are considered â\x80\x9cSignificantâ\x80\x9d. Oral pr...",2018
1,mentioned more than 60/169 reasons. The reasons could The ethics of clinical applications of germline genome becategorisedinto:(i)qualityoflifeofaffectedindividuals; modification:a systematic rev...,2018
2,implications. Asystematicoverviewofreasonsforbeingin Enablinginformed opinionsabout germline editingamong favouroragainstgermlinegenomemodificationismissing. the general public: a pilot study We ...,2018
3,measurement this ratio flipped. The PRISM-IMPACT study: What are the hopes and Discussion/conclusion: Our pilot study demonstrates a expectations of families and health care professionals signifi...,2018
4,"Cancer Centre, Sydney Childrenâ\x80\x99s Hospital, Randwick, NSW, Informedconsentinahumangermlinegeneeditingstudy- Australia, 3Hereditary Cancer Centre, Prince of Wales ethical issues Hospital, S...",2018


In [97]:
def batch_by_year(input):                                                     # create a function that
    years = texts_year_only['Year'].drop_duplicates().sort_values().to_list() # gets a sorted list of all unique year values
    for year in years:                                                        # then iterates over that list to create a temp
        temp_df = texts_year_only[texts_year_only.Year == year]               # data frame with only the rows matching each year
        temp_df.to_csv('..\\results\\batch_' + str(year) + '.csv')            # saving the year-filtered data .csv in '\results'


In [96]:
batch_by_year(texts_year_only)                 # run that newly defined function on our texts

In [ ]:
def bag_of_words_analysis(input, how_many):     # define a 'bag of words' function with 2 arguments, an input and a quantity 
    holding_string = ""                                                        # that creates a temporary variable
    for text in input['Text']:                                                 # looks at the 'Text' column for the input
        holding_string += text                                                 # fills up the temp variable with the text
    holding_string = word_tokenize(holding_string)                             # word tokenises that text
    holding_string = [word.lower() for word in holding_string]                 # remove uppercase letters
    holding_string = [w.translate(table_punctuation) for w in holding_string]  # removes punctuation
    holding_string = (list(filter(lambda x: x, holding_string)))               # removes andy empty strings
    holding_string = [token for token in holding_string if not token.isdigit()]  # removes digits
    holding_string = [token for token in holding_string if token not in stop_words]  # removes stopwords
    holding_string = [porter.stem(token) for token in holding_string]                # stems the word-tokens
    list_for_count = []                                                              # and creates an empty list
    for token in holding_string:                                         # then iterates over the tokens
        list_for_count.append(token)                                     # appending them to the list
    counts = Counter(list_for_count)                                     # applies the Counter function imported earlier 
    return counts.most_common(how_many)                                  # and returns the tokens with highest counts 
                                                                         # up to the quantity specified as an argument

## Save the consolidated output as .csv

Having imported, consolidated, tidied and checked everything, I want to save the output in a new .csv file. It is important to use a good name for the file, because bad file names are the bane of my existance. 

For simplicity sake, I will also create a new data frame containing only those rows for which the 'Text' column contains one of the keywords of interest, check its length and save it as a new .csv file with a good name. 

In [ ]:
type(no_null_texts)                          # Let's just double check what kind of a thing 'no_null_texts' is
                                             # This lets us know what kind of write-out-to-csv function we need.

In [ ]:
no_null_texts


In [ ]:
no_null_texts.to_csv('..\\output\\all_abstracts_no_null_texts.csv')  # write out the data frame to a .csv, with a useful name
                                                                     # which clarifies this is ALL abstracts with non-null texts

## Manually check the saved .csv files

You may want to go and check that the two files you have created here have been created and saved correctly. You may even want to open them up and have a nosy through them to see what they look like. 

The next notebook picks up where this leaves off, by importing those files and working with them to produce some stats that help explore the research question. 